In [7]:
%matplotlib inline
%reset

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from py2neo import Graph,authenticate,Relationship,Node
import networkx as nx
import community
import numpy as np
from itertools import groupby
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from PyDictionary import PyDictionary
from collections import Counter
from datetime import datetime
import pandas as pd
from pymongo import MongoClient
import json
import pickle

import ijson

import sqlite3



folder=''

G=Graph()

client = MongoClient()
db=client.muzmatch

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [8]:
data='../repos/'

Load the dictionaries
------------------------
*d.pkl is the dict of all users

*dUK.pkl is the dictionary of users in the UK

This includes the following fields:

*Gender

*Age

*Ethnicity

*Long Description

*Status Message

*Number of Likes


In [9]:
d=pickle.load(open(data+'d.pkl','rb'))
dUK=pickle.load(open(data+'dUK.pkl','rb'))

This creates a new collection in the mongo MuzMatch database, which removes all the liked users who are no longer active.

In [10]:
def trim_likes(d,doc):

    likes=doc['who_i_liked']
    trimmed_likes=[]
    date=[]
    for l in likes:
        mid=l['user_id1']
        date.append(l['timeStamp'].split(' ')[0])
        if mid in d.keys():
            trimmed_likes.append(mid)
    doc['who_i_liked']=trimmed_likes
    
    doc['activeSince']=min(date)
    return doc

In [13]:
collections=[db.n1,db.n2,db.n3,db.n4,db.n5]


In [14]:
db.trim.drop()
trim=db.trim

In [15]:
for coll in collections:
    for doc in coll.find({},{'memberID':1,"who_i_liked":1}):
        if 'who_i_liked' in doc.keys():
            doc=trim_likes(d,doc)
            if doc['who_i_liked']!=[] and doc['who_i_liked'] is not None:#ensure that user only likes active users, otherwise assume user effectively inactive
                
                trim.insert_one(doc)

In [20]:
print('we have %s records'%db.trim.count())

we have 41466 records


Pandify
---------------

In [16]:
df=pd.DataFrame.from_dict(d,orient='index')
dfUK=pd.DataFrame.from_dict(dUK,orient='index')

In [17]:
df=df[df.age>0]

In [18]:
dfUK=dfUK[dfUK.age>0]

Gender Stats
--------------

In [ ]:
plt.xkcd()
df.gender.value_counts().plot(kind='bar')
df.gender.value_counts()['M']/df.gender.value_counts()['F']

In [ ]:
plt.xkcd()
dfUK.gender.value_counts().plot(kind='bar')
dfUK.gender.value_counts()['M']/dfUK.gender.value_counts()['F']

Ethnicity Stats
---------------

In [ ]:
df.ethnic.value_counts()
dfUK.ethnic.value_counts()


In [ ]:
dth=pd.DataFrame(df.groupby(['ethnic','gender']).size())
dthUK=pd.DataFrame(dfUK.groupby(['ethnic','gender']).size())


In [ ]:
plt.xkcd()
df.ethnic.value_counts().plot(kind='pie',figsize=(6, 6))

In [ ]:
plt.xkcd()
dfUK.ethnic.value_counts().plot(kind='pie',figsize=(6, 6))

Age Stats
---------

In [ ]:
plt.xkcd()
df.age.hist(bins=[20,23,26,29, 32,35,38,41,45]).figure

In [ ]:
plt.xkcd()
dfUK.age.hist(bins=[20,23,26,29, 32,35,38,41,45]).figure

In [ ]:
plt.xkcd()

countsM,bins=np.histogram(df[df.gender=='M'].age,bins=[20,25,30,35,40,45,50,55,60,65,70])
countsF,bins=np.histogram(df[df.gender=='F'].age)

fig=plt.figure()
ax=fig.add_subplot(111)

ax.bar(bins[:-1],countsM/max(countsM),width=3)
ax.bar(bins[:-1]+6,countsF/max(countsF),color='r',width=3)
plt.xlabel('age')
plt.ylabel('normalised frequency')

In [ ]:
plt.xkcd()

countsM,bins=np.histogram(dfUK[dfUK.gender=='M'].age)
countsF,bins=np.histogram(dfUK[dfUK.gender=='F'].age)

fig=plt.figure()
ax=fig.add_subplot(111)

ax.bar(bins[:-1],countsM/max(countsM))
ax.bar(bins[:-1]+1,countsF/max(countsF),color='r')
plt.xlabel('age')
plt.ylabel('normalised frequency')

Swipe Stats
-------------

In [ ]:
dlikesM=pickle.load(open(data+'dlikesM.pkl','rb'))
dlikesF=pickle.load(open(data+'dlikesF.pkl','rb'))


In [ ]:
plt.xkcd()
countsM,bins=np.histogram(np.log10(list(dlikesM.values())),bins=5)
countsF,binsF=np.histogram(np.log10(list(dlikesF.values())),bins=bins)

fig=plt.figure()
ax=fig.add_subplot(111)
ax.bar(bins[:-1],countsM,width=0.5)
ax.bar(bins[:-1],countsF,color='r',width=0.5)
plt.xlabel('Logarithmic number of likes (base 10)')
plt.ylabel('counts')

In [ ]:
A=np.trapz(list(dlikesM.values()),dx=1)
y=list(dlikesM.values())
delta=10
l=0
B=0
while B<A*0.5:
    B=np.trapz(y[0:l],dx=5)
    l+=delta

print('%s likes make up 50 percent of the area under the curve'%y[l])

In [ ]:
10**0.85

In [ ]:
A=np.trapz(list(dlikesF.values()),dx=5)
y=list(dlikesF.values())
delta=10
l=0
B=0
while B<A*0.5:
    B=np.trapz(y[0:l],dx=5)
    l+=delta
print('%s likes make up 50 percent of the area under the curve'%y[l])    

So we keep those who like up to 900 people.

In [ ]:
print('max Men likes %s\n'%max(dlikesM.values()))
print('max Women likes %s\n'%max(dlikesF.values()))

print('min Men likes %s\n'%min(dlikesM.values()))
print('min Women likes %s\n'%min(dlikesF.values()))

print('mean Men likes %s\n'%np.mean(list(dlikesM.values())))
print('mean Women likes %s\n'%np.mean(list(dlikesF.values())))

In [ ]:
today='2017-04-01'


In [ ]:
#this function adds number of swipes to each user
trim=db.trim
dlikesMUK={}
dlikesFUK={}
for key in dUK.keys():
    t=trim.find({'memberID':key},{"who_i_liked":1})
    a=trim.find({'memberID':key},{"activeSince":1})
    try:
        dUK[key]['activeSince']= a[0]['activeSince']
    except:
        continue
            
    for tt in t:
        if 'who_i_liked' in tt.keys():
            likes=len(tt['who_i_liked' ])
            if d[key]['gender']=='M':
                dlikesMUK[key]=likes
            else:
                dlikesFUK[key]=likes
                    


In [ ]:
pickle.dump(dlikesMUK,open(data+'dlikesMUK.pkl','wb'))
pickle.dump(dlikesFUK,open(data+'dlikesFUK.pkl','wb'))

In [ ]:
plt.xkcd()
countsM,bins=np.histogram(np.log10(list(dlikesMUK.values())),bins=5)
countsF,binsF=np.histogram(np.log10(list(dlikesFUK.values())),bins=bins)

fig=plt.figure()
ax=fig.add_subplot(111)
ax.bar(bins[:-1],countsM,width=0.5)
ax.bar(bins[:-1],countsF,color='r',width=0.25)
plt.xlabel('Logarithmic number of likes (base 10)')
plt.ylabel('counts')

In [ ]:
print('max Men likes %s\n'%max(dlikesMUK.values()))
print('max Women likes %s\n'%max(dlikesFUK.values()))

print('min Men likes %s\n'%min(dlikesMUK.values()))
print('min Women likes %s\n'%min(dlikesFUK.values()))

print('mean Men likes %s\n'%np.mean(list(dlikesMUK.values())))
print('mean Women likes %s\n'%np.mean(list(dlikesFUK.values())))

In [ ]:
bins[:-1],countsM

10**2.4

In [ ]:
def get_age_age(d,dictM,dictF):
    for key in d.keys():
        t=trim.find({'memberID':key},{'who_i_liked':1})
        for tt in t:
            if d[key]['gender']=='M':
                l=[]
                if 'who_i_liked' in tt.keys():
                    for mid in tt['who_i_liked']:
                        if mid in d.keys(): #filtered out nonUK people and international lurve is a thing
                            l.append(d[mid]['age'])
                dictM[d[key]['age']]=l

            if d[key]['gender']=='F':
                if 'who_i_liked' in tt.keys():
                    l=[]
                    for mid in tt['who_i_liked']:
                        if mid in d.keys():
                            l.append(d[mid]['age'])
                dictF[d[key]['age']]=l
    return dictM,dictF

In [ ]:
#age vs age swipes for men and women
agesM={}
agesF={}
agesM,agesF=get_age_age(d,agesM,agesF)



In [ ]:
pickle.dump(agesM,open(data+'agesM.pkl','wb'))
pickle.dump(agesF,open(data+'agesF.pkl','wb'))

In [ ]:
agesM=pickle.load(open(data+'agesM.pkl','rb'))
agesF=pickle.load(open(data+'agesF.pkl','rb'))

In [ ]:
agesMUK={}
agesFUK={}
agesMUK,agesFUK=get_age_age(dUK,agesMUK,agesFUK)

In [ ]:
pickle.dump(agesMUK,open(data+'agesMUK.pkl','wb'))
pickle.dump(agesFUK,open(data+'agesFUK.pkl','wb'))

In [ ]:
def get_trends(d,dict1):
    for age in dict1.keys():
        ageMean=np.mean(dict1[age])
        ageSdev=np.std(dict1[age])
        z=Counter()
        for a in dict1[age]:
            z[a]+=1
        d[age]={}
        d[age]['stats']=(ageMean,ageSdev)
        d[age]['counts']=z
    return d

In [ ]:
agesMUK=pickle.load(open(data+'agesMUK.pkl','rb'))
agesFUK=pickle.load(open(data+'agesFUK.pkl','rb'))

In [ ]:
agesTrendF={}
agesTrendF=get_trends(agesTrendF,agesF)
agesTrendM={}
agesTrendM=get_trends(agesTrendM,agesM)

agesTrendFUK={}
agesTrendFUK=get_trends(agesTrendFUK,agesFUK)
agesTrendMUK={}
agesTrendMUK=get_trends(agesTrendMUK,agesMUK)

    

In [ ]:
pickle.dump(agesTrendM,open(data+'agesTrendM.pkl','wb'))
pickle.dump(agesTrendF,open(data+'agesFTrend.pkl','wb'))

pickle.dump(agesTrendMUK,open(data+'agesTrendMUK.pkl','wb'))
pickle.dump(agesTrendFUK,open(data+'agesFTrendUK.pkl','wb'))

In [ ]:
def plt_scatter(d):
    plt.xkcd()
    for age in d.keys():
        if age>0 and age<60:
            y=list(d[age]['counts'].keys())
            y=[yy for yy in y if yy>0 and yy<60]
            s=list(d[age]['counts'].values())
            plt.scatter([age]*len(y),y,s=s)
        
def plt_trend(d):
    x=[]
    m=[]
    mp=[]
    mm=[]

    for age in d.keys():
        if age>20 and age<60:
            x.append(age)
            m.append(d[age]['stats'][0])
            mp.append(d[age]['stats'][0]+d[age]['stats'][1])
            mm.append(d[age]['stats'][0]-d[age]['stats'][1])
    s,b = np.polyfit(x, m, 1)
    plt.plot(x,[s*float(c)+b for c in x],'r')
    s,b = np.polyfit(x, mp, 1)
    plt.plot(x,[s*float(c)+b for c in x],'r--')
    s,b = np.polyfit(x, mm, 1)
    plt.plot(x,[s*float(c)+b for c in x],'r--')

    print('slope is %s with an intercept of %s'%(s,b))

In [ ]:
plt_scatter(agesTrendF)
plt_trend(agesTrendF)
plt.xlabel('Female Age')
plt.ylabel('Male Age')
plt.axis([20,60,20,60])




In [ ]:
plt_scatter(agesTrendM)
plt_trend(agesTrendM)
plt.ylabel('Female Age')
plt.xlabel('Male Age')
plt.axis([20,60,20,60])



    

In [ ]:
plt_scatter(agesTrendMUK)
plt_trend(agesTrendMUK)
plt.ylabel('Female Age')
plt.xlabel('Male Age')
plt.axis([20,60,20,60])


In [ ]:
plt_scatter(agesTrendFUK)
plt_trend(agesTrendFUK)
plt.xlabel('Female Age')
plt.ylabel('Male Age')
plt.axis([20,60,20,60])



In [ ]:
d2=pickle.load(open(data+'d2.pkl','rb'))
bad_keys=[]
for key in d2.keys():
    if d2[key]['activeSince']==0:
        bad_keys.append(key)

In [ ]:
trim=db.trim
for key in bad_keys:
    t=trim.find({'memberID':key},{"activeSince":1})
    try:
        d2[key]['activeSince']=t[0]['activeSince']
    except:
        d2[key]['activeSince']=0
    
    

In [ ]:
bad_keys=[]
for key in d2.keys():
    if d2[key]['activeSince']==0:
        bad_keys.append(key)
        
len(bad_keys)

In [ ]:
today=datetime.strptime('2017-04-28','%Y-%m-%d')
dUK2={}
for key in dUK.keys():
    if key in d2.keys() and 'activeSince' in d2[key].keys():
        a=d2[key]['activeSince']
        if a==0:
            a=today
        else:
            a=datetime.strptime(a,'%Y-%m-%d')
        a=(today-a).days
        if a==0:
            a=0
        dUK2[key]=dUK[key]
        dUK2[key]['daysActive']=a

In [ ]:
pickle.dump(dUK2,open(data+'dUK2.pkl','wb'))

NLP Magic
=====

In [ ]:
pUK=pickle.load(open(data+'pUK.pkl','rb'))

In [ ]:
#len(pUK.ix['100024']['real'])
#dUK['100024']['SM']
pUK

In [ ]:
def get_len(sm):
    if sm is not None:
        return len(sm)
    else:
        return 0

In [ ]:
pUK['SMlen']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['SMlen']=pUK['SM'].apply(get_len)

pUK['LDlen']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['LDlen']=pUK['LD'].apply(get_len)

In [ ]:
dreal=pickle.load(open(data+'dreal.pkl','rb'))

In [ ]:
def get_reals(d):
    return dreal[d]['realLikes']

In [ ]:
pUK['real']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['real']=pUK.index.map(get_reals)


In [ ]:
def real_len(d):
    if d==0:
        return 0
    else:
        return len(d)

In [ ]:
pUK['realLen']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['realLen']=pUK['real'].apply(real_len)


In [ ]:
max(pUK['realLen'])

In [ ]:
pUK['SM'][pUK['SM'].isnull()]='XXXXMONKEYSXXXX'

In [ ]:
pUK['LD'][pUK['LD'].isnull()]='XXXXMONKEYSXXXX'

In [ ]:
tf = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english',max_features=50)
tfLD = TfidfVectorizer(analyzer='word', min_df=0, stop_words='english',max_features=500)



tfidf_SMUK = tf.fit_transform(pUK['SM'])
tfidf_LDUK = tfLD.fit_transform(pUK['LD'])

In [ ]:
feature_names = tf.get_feature_names()

i = 0
top_words = {}
max_words=1
for row in tfidf_SMUK.todense():
    task = row.tolist()[0]
    top_words[i] = [feature_names[n] for n in list(range(0, len(task))) if task[n] != 0]

    i += 1
top_SM=top_words

top_SMList=list(set([l for b in list(top_SM.values()) for l in b]))

feature_names = tfLD.get_feature_names()


i = 0
top_words = {}
max_words=10
for row in tfidf_LDUK.todense():
    task = row.tolist()[0]
    top_words[i] = [feature_names[n] for n in list(range(0, len(task))) if task[n] != 0]

    i += 1
top_LD=top_words
top_LDList=list(set([l for b in list(top_LD.values()) for l in b]))



In [ ]:
top_SM=top_SMList
top_LD=top_LDList

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud().generate(' '.join(smt))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
wordcloud = WordCloud().generate(' '.join(ldt))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

In [ ]:
dlikesM=pickle.load(open(data+'dlikesM.pkl','rb'))
dlikesF=pickle.load(open(data+'dlikesF.pkl','rb'))

In [ ]:
dlikes={**dlikesM,**dlikesF}

In [ ]:
pUK['likes']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)

In [ ]:
def get_active_since(mid):
    if 'daysActive' in dUK2[mid].keys():
        return dUK2[mid]['daysActive']
    else:
        return 0

In [ ]:
pUK['activeSince']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['activeSince']=pUK.index.map(get_active_since)

In [ ]:
pUK=pUK[pUK['activeSince']!=0]

In [ ]:
def add_likes(mid):
    if mid in dlikes.keys():
        return dlikes[mid]
    else:
        return 0

In [ ]:
pUK['likes']=pUK.index.map(add_likes)

In [ ]:
def filter_top(t):
    words=[]
    if len(t)>0:
        tt=t.split(' ')

       
        for ttt in tt:
            if ttt in top_SM:
                words.append(ttt)
        return words
def filter_topLD(t):
    words=[]
    if len(t)>0:
        tt=t.split(' ')

       
        for ttt in tt:
            if ttt in top_LD:
                words.append(ttt)
        return words

In [ ]:
c,b=np.histogram(pUK['activeSince'])
plt.plot(b[:-1],c)
plt.xlabel('Days Active')
plt.ylabel('Counts')

In [ ]:
c,p=np.histogram(pUK['realLen'][(pUK['activeSince']<500)])
plt.plot(b[:-1],c)
plt.xlabel('Number of Likes')
plt.ylabel('Counts')

In [ ]:
c,p=np.histogram(pUK['likes'][(pUK['activeSince']<500)])
plt.plot(b[:-1],c)
plt.xlabel('Number of Likes')
plt.ylabel('Counts')

In [ ]:
len(pUK[pUK['realLen']<200])

In [ ]:
pUK['SM']=pUK['SM'].apply(filter_top)
pUK['LD']=pUK['LD'].apply(filter_topLD)

In [ ]:
pUK.to_pickle(data+'pUK2.pkl')

In [ ]:
pop=G.data('match ()-[r]->(n) return n.mid as mid,count(r) as num')

In [ ]:
pops={}
for item in pop:
    pops[item['mid']]=item['num']

In [ ]:
def get_pop(p):
    if p in pops.keys():
        return pops[p]
    else:
        return 0

In [ ]:
pUK['pop']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['pop']=pUK.index.map(get_pop)
pUK['popNorm']=round(pUK['pop']/pUK['activeSince'])

In [ ]:
pUK.to_pickle(data+'pUK2.pkl')

In [ ]:
counts,bins=np.histogram(pUK['pop'],bins=[50,100,150,200,250,300,350,400,450,500,550])
pcounts=counts/sum(counts)*100
min_likes=100

In [ ]:
countsPop,binsPop=np.histogram(pUK['popNorm'])
plt.plot(binsPop[:-1],countsPop)


In [ ]:
np.std(pUK['popNorm'][pUK['gender']=='M'])+np.sqrt(np.mean(pUK['popNorm'][pUK['gender']=='M']))



In [ ]:
np.std(pUK['popNorm'][pUK['gender']=='F'])+np.sqrt(np.mean(pUK['popNorm'][pUK['gender']=='F']))

In [ ]:
def get_target(p):
    gender=p['gender']
    pop=p['popNorm']
    if gender=='M' and pop>9:
        return 1
    elif gender=='F' and pop>80:
        return 1
    else:
        return 0

In [ ]:
pUK['target']=pd.Series(np.random.randn(len(pUK)), index=pUK.index)
pUK['target']=pUK.apply(get_target,axis=1)

In [ ]:
np.mean(pUK['popNorm'][pUK['gender']=='F'])

In [ ]:
def concat_SM(sm):
    new_sm=[]
    if sm is not None:
        if len(sm)!=0:
            for t in sm:
                new_sm.append('SM_'+t)
    return new_sm
def concat_LD(sm):
    new_sm=[]
    if sm is not None:
        if len(sm)!=0:
            for t in sm:
                new_sm.append('LD_'+t)
    return new_sm
                

In [ ]:
pUK['SM']=pUK['SM'].apply(concat_SM)

In [ ]:
pUK['LD']=pUK['LD'].apply(concat_LD)

In [ ]:
colSM=pUK['SM'].str.join('|').str.get_dummies()
colLD=pUK['LD'].str.join('|').str.get_dummies()
colEth=pUK['ethnic'].str.get_dummies()
countAge,binsAge=np.histogram(pUK['age'],bins=[20,25,30,35,40,45])
countSM,binsSM=np.histogram(pUK['SMlen'],bins=20)
countLD,binsLD=np.histogram(pUK['LDlen'],bins=20)


In [ ]:
countlikes,binslikes=np.histogram(pUK['likes'])


In [ ]:
colgender=pUK['gender'].str.get_dummies()

In [ ]:
def get_bin(data,bins):
    for b in bins:
        if data<b:
            return round(b)

In [ ]:
pUK['age']=pUK['age'].apply(get_bin,args=(binsAge,))

In [ ]:
pUK['SMlen']=pUK['SMlen'].apply(get_bin,args=(binsSM,))

In [ ]:
pUK['LDlen']=pUK['LDlen'].apply(get_bin,args=(binsLD,))

In [ ]:
pUK['likes']=pUK['likes'].apply(get_bin,args=(binslikes,))

In [ ]:
pUK['age']=pUK['age'].map(str)

In [ ]:
def append_string(sm,st):
    return st+sm

In [ ]:
colage=pUK['age'].str.get_dummies()
colage.columns=['age_'+a for a in colage.columns]


In [ ]:
pUK['SMlen']=pUK['SMlen'].map(str)
colSMlen=pUK['SMlen'].str.get_dummies()
colSMlen.columns=['smLen_'+ l for l in colSMlen.columns]

In [ ]:
pUK['LDlen']=pUK['LDlen'].map(str)
colLDlen=pUK['LDlen'].str.get_dummies()
colLDlen.columns=['ldLen_'+l for l in colLDlen.columns]

In [ ]:
pUK['likes']=pUK['likes'].map(str)
colLikes=pUK['likes'].str.get_dummies()
colLikes.columns=['likes_'+l for l in colLikes.columns]

In [ ]:
pUK['activeSince']=pUK['activeSince'].map(str)
colActive=pUK['activeSince'].str.get_dummies()
colActive.columns=['ActiveDays_'+l for l in colActive.columns]

In [ ]:
pUKM=pUK[pUK['gender']=='F']
colSMM=colSM.ix[[i for i in colSM.index if i in pUKM.index]]
colLDM=colLD.ix[[i for i in colLD.index if i in pUKM.index]]
colEthM=colEth.ix[[i for i in colEth.index if i in pUKM.index]]
colgenderM=colgender.ix[[i for i in colgender.index if i in pUKM.index]]
colLikesM=colLikes.ix[[i for i in colLikes.index if i in pUKM.index]]
colLDlenM=colLDlen.ix[[i for i in colLDlen.index if i in pUKM.index]]
colSMlenM=colSMlen.ix[[i for i in colSMlen.index if i in pUKM.index]]
colageM=colage.ix[[i for i in colage.index if i in pUKM.index]]
colActiveM=colActive.ix[[i for i in colActive.index if i in pUKM.index]]


In [ ]:
pML=pd.concat([colActiveM,colEthM,colageM,colSMlenM,colLDlenM],axis=1)

In [ ]:
colActiveM

Machine Learning Baby!
============

In [ ]:
from sklearn.metrics import roc_curve, auc
import pandas as pd
from sklearn_pandas import DataFrameMapper, cross_val_score
import sklearn
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
#model=linear_model.LogisticRegression()
model=RandomForestClassifier(n_jobs=-1,max_depth=5,random_state=42,n_estimators=500,max_features = "auto", min_samples_leaf = 50,oob_score=True)

In [ ]:
pML=pML.reset_index()
pML=pML.drop('index',axis=1)

In [ ]:
cv = StratifiedKFold(np.array(pUK.target[pUK['gender']=='F']), n_folds=6)
y=np.array(pUK.target)
X=pML

In [ ]:
from sklearn.metrics import roc_auc_score

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []
for i, (train, test) in enumerate(cv):
    probas_ = model.fit(X.ix[list(train)], y[train]).predict_proba(X.ix[list(test)])
    # Compute ROC curve and area the curve
   
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    
    plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
#plt.legend(loc="right")
plt.show()

In [ ]:
importances=model.feature_importances_
indices = np.argsort(importances)[::-1]
names=pML.columns


In [ ]:
for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, names[indices[f]], importances[indices[f]]))

In [ ]:
pUK.sort(columns=['popNorm'],ascending=False)

In [ ]:
d2['74743']

Notes
------
Given the huge number of nodes 

In [ ]:
graph_edges=pd.DataFrame(G.data("MATCH (m)-[r]->(n) RETURN m.mid,n.mid"))
edges=list(zip(graph_edges['m.mid'].tolist(),graph_edges['n.mid'].tolist()))

In [ ]:
G2=nx.Graph()
def nodes_connected(u, v):
    return u in G2.neighbors(v)

for edge in edges:
    node1 = edge[0]
    node2 = edge[1]
    if node1 and node2 in G2.nodes():
        if not nodes_connected(node1, node2):
            c = 0.1

            G2.add_edge(node1, node2, weight=c)
        else:
            c += 0.1
            G2.add_edge(node1, node2, weight=c)

    else:
        if node1 not in G2.nodes():
            G2.add_node(node1)
        if node2 not in G2.nodes():
            G2.add_node(node2)
        c = 0.1
        G2.add_edge(node1, node2, weight=c)

In [ ]:
partitions1 = community.best_partition(G2)
partitions = [(k, v) for k, v in partitions1.items()]

sorted_partitions = sorted(partitions, key=lambda tup: tup[1])
communities = groupby(sorted_partitions, key=lambda tup: tup[1])

mod = community.modularity(partitions1,G2)

# Plot, color nodes using community structure
values = [partitions1.get(node) for node in G2.nodes()]
nx.draw_spring(G2, cmap=plt.get_cmap('jet'), node_color = values, node_size=30, with_labels=False)
plt.show()

In [ ]:
comm={}
for c, u in communities:
    print(c)
    u = list(u)
    u = [y[0] for y in u]
    comm[c] = u
    print('Community %s has %s members\n' % (c, len(u)))
    #file.write('----------------------------\n')

    #for uu in u:
     #   file.write('%s\n' % ' '.join(uu.split('_')))

    #file.write('-----------------------------\n')

In [ ]:
#invert dictionary
peeps={}
for key in comm.keys():
    people=comm[key]
    for p in people:
        peeps[p]=key

In [ ]:
comm_ethnic={}
for key in comm.keys():
    l=[]
    for p in comm[key]:
        eth=d[p]['ethnic']
        l.append(eth)
    comm_ethnic[key]=l

comm_age={}

for key in comm.keys():
    l=[]
    for p in comm[key]:
        age=d[p]['age']
        l.append(age)
    comm_age[key]=(np.mean(l),np.sqrt(np.var(l)))

In [ ]:
comm_age_ethnic={}
for key in comm.keys():
    comm_age_ethnic[key]={}
    for p in comm[key]:

        typ=d[p]['ethnic']
        if typ in comm_age_ethnic[key].keys():
            age=d[p]['age']
            l.append(age)
        else:
            age=d[p]['age']
            l=[age]
        comm_age_ethnic[key][typ]=l
    for typ in comm_age_ethnic[key].keys():
        comm_age_ethnic[key][typ]=(np.mean(comm_age_ethnic[key][typ]),np.sqrt(np.var(comm_age_ethnic[key][typ])))
    comm_age_ethnic[key]['allStats']=comm_age[key]
            

In [ ]:
comm_age_ethnic_gender={}
for key in comm.keys():
    comm_age_ethnic_gender[key]={}
    for p in comm[key]:

        typ=d[p]['ethnic']
        g=d[p]['gender']
        if g=='F':
            if 'F' not in comm_age_ethnic_gender[key].keys():
                comm_age_ethnic_gender[key]['F']={}
            if typ in comm_age_ethnic_gender[key]['F'].keys():
                age=d[p]['age']
                l.append(age)
            else:
                age=d[p]['age']
                l=[age]
               
                
            comm_age_ethnic_gender[key]['F'][typ]=l
        else:
            if 'M' not in comm_age_ethnic_gender[key].keys():
                
                comm_age_ethnic_gender[key]['M']={}
                
            
            
            if typ in comm_age_ethnic_gender[key]['M'].keys():
                age=d[p]['age']
                l.append(age)
            else:
                age=d[p]['age']
                l=[age]

            comm_age_ethnic_gender[key]['M'][typ]=l 

                   
    for gen in comm_age_ethnic_gender[key].keys():
        for typ in comm_age_ethnic_gender[key][gen].keys():
            l=[ll for ll in comm_age_ethnic_gender[key][gen][typ] if type(ll)==int]
            comm_age_ethnic_gender[key][gen][typ]=(np.mean(l),np.sqrt(np.var(l)))
        #comm_age_ethnic_gender[key][gen]['allStats']=comm_age[key][gen]
          

In [ ]:
comm_age_ethnic_gender

In [ ]:
comm_avg_pop={}

for key in comm.keys():
    typ=Counter()
    comm_avg_pop[key]={}
    for p in comm[key]:
        if p in pUK.index:
            popularity=pUK.ix[p]['popNorm']
            age=d[p]['age']
            eth=d[p]['ethnic']
            g=d[p]['gender']
            sm=pUK.ix[p]['SM']
            ld=pUK.ix[p]['LD']
            likes=pUK.ix[p]['likes']
            ldLen=pUK.ix[p]['LDlen']
            smLen=pUK.ix[p]['SMlen']
            typ[eth]+=1
            if g in comm_avg_pop[key].keys():
                comm_avg_pop[key][g]['SM'].append(sm)
                comm_avg_pop[key][g]['LD'].append(ld)
                comm_avg_pop[key][g]['ldLen'].append(ldLen)
                comm_avg_pop[key][g]['smLen'].append(smLen)
            else:
                comm_avg_pop[key][g]={}
                comm_avg_pop[key][g]['SM']=[sm]
                comm_avg_pop[key][g]['LD']=[ld]
                comm_avg_pop[key][g]['ldLen']=[ldLen]
                comm_avg_pop[key][g]['smLen']=[smLen]
                    
                
                
            
            
            
            
            
            
    
    

In [ ]:
comm_avg_pop